Reinforcement Learning for Large Language Models
Winter 23/24 Semester
Final Group Project
Kateryna Smykovska, Jakob Schmitter, Suvi Lehtosalo, Megan Horikawa

This notebook was made by Megan Horikawa

[NLI - 7b Notebook](https://colab.research.google.com/drive/1I4fAi-c0WCogzFD4nz9ipn4z1yMjRP-V?usp=sharing)

In [1]:
!pip install transformers
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece
!pip install torch
!pip install datasets
!pip install evaluate
!pip install huggingface_hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00


In [2]:
from transformers import LlamaForCausalLM, LlamaTokenizer, AutoTokenizer
import torch


import datasets
import evaluate
import numpy as np

from google.colab import userdata
my_secret_key = userdata.get('HF_TOKEN')

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", token=my_secret_key)

model = LlamaForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    load_in_4bit=True,
    device_map="auto",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    token = my_secret_key
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [3]:
# Load Dataset for NLI task. Superglue subdataset rte was chosen
rte_dataset = datasets.load_dataset("super_glue", "rte")

# will use validation subset as it contains but entailment and non-entailment classes unlike test set

dataset = rte_dataset['validation']

# not entailment = 1 entailment = 0

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [4]:
# define log likelihood function (taken from homework with small changes for llama)

def get_log_prob_of_completion(
        model,
        tokenizer,
        prompt,
        completion,
        device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
):
        """
        Convenience function for computing the log probability of a completion
        given a prompt.
        """
        # tokenize the prompt and the completion
        # truncate so as to fit into to maximal context window of llama2
        # which is 2048 tokens
        input_ids = tokenizer(
                prompt + completion,
                return_tensors='pt',
                truncation=True,
                max_length=2048,
        )['input_ids'].to(device)

        # separately tokenize prompt
        # so as to access the logits for the completion only
        # when scoring the completion
        input_ids_prompt = tokenizer(
                prompt,
                return_tensors='pt',
                truncation=True,
                max_length=2048
        )['input_ids'].to(device)

        # create attention mask and position ids
        attention_mask = (input_ids != tokenizer.eos_token_id).to(dtype=torch.int64)
        position_ids = attention_mask.cumsum(-1)-1
        # get the logits for the completion
        with torch.no_grad():
                out = model(
                        input_ids=input_ids,
                        attention_mask=attention_mask,
                        position_ids=position_ids
                )

        # get the logits of the completion
        # for that, make a tensor of the logits
        # for the completion only
        # in particular, we shift the indices by one to the left to access logits of the
        # actual sequence tokens
        logits_completion = out.logits[:, :-1]
        logits_completion = logits_completion.squeeze()
        # get the log probabilities for the completion
        log_probs = torch.nn.functional.log_softmax(
                logits_completion,
                dim=-1
        )
        # retrieve the logit corresponding to the actual completion tokens
        try:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].squeeze().unsqueeze(-1)
                )
        except:
                log_completion_tokens = log_probs.gather(
                        dim=-1,
                        index=input_ids[:, 1:].unsqueeze(-1)
                )

        continuationConditionalLogProbs = log_completion_tokens[
                (input_ids_prompt.shape[-1]-1):
        ]
        completion_log_prob = torch.mean(
                continuationConditionalLogProbs
        ).cpu()

        return completion_log_prob

In [5]:
# using method of Homework 2 #3

#iterate over the dataset:

results = []
prompt_list = []
correct_answer_list =[]
log_ps_correct_option = []
log_ps_incorrect_option = []

for item in dataset:
  # as done in homework 2 the prompt is both premise and hypothesis concatenated.
  prompt = item['premise'] + " " + item['hypothesis']
  # get classification of item as either entailment (0) or non-entailment(1)
  category = item['label']

  correct = ''
  incorrect = ''

  # set correct classification based on label
  if category == 1:
    correct = 'non-entailment'
    incorrect = 'entailment'

  else:
    correct = 'entailment'
    incorrect = 'non-entailment'

  prompt_list.append(prompt)
  correct_answer_list.append(correct)

  #compute the lob probabilities for both entailment and non-entailment classes

  log_p_correct = get_log_prob_of_completion(
      model,
      tokenizer,
      prompt,
      correct
  )
  # append to the correct list
  log_ps_correct_option.append(log_p_correct)

  log_p_incorrect = get_log_prob_of_completion(
      model,
      tokenizer,
      prompt,
      incorrect
  )

  # append to the incorrect list
  log_ps_incorrect_option.append(log_p_incorrect)



In [6]:
# lets load things into pandas
import pandas as pd

df = pd.DataFrame(list(zip(prompt_list,correct_answer_list,log_ps_correct_option,log_ps_incorrect_option)), columns = ['prompt','entailment/non-entailment','log_prob_correct', 'log_prob_incorrect'])


# evaluate the log probabilities
# check whether the log probaility of the correct answer is higher than the
# incorrect answer and append to the results list
df['correct_prediction'] = df['log_prob_correct']> df['log_prob_incorrect']


df.head(20)


,prompt,entailment/non-entailment,log_prob_correct,log_prob_incorrect,correct_prediction
0,"Dana Reeve, the widow of the actor Christopher...",non-entailment,tensor(-6.5984),tensor(-7.8941),True
1,"Yet, we now are discovering that antibiotics a...",entailment,tensor(-8.5193),tensor(-7.2770),False
2,Cairo is now home to some 15 million people - ...,non-entailment,tensor(-7.1115),tensor(-8.7469),True
3,"The Amish community in Pennsylvania, which num...",non-entailment,tensor(-7.1602),tensor(-9.5456),True
4,Security forces were on high alert after an el...,entailment,tensor(-9.9538),tensor(-7.0786),False
5,"In 1979, the leaders signed the Egypt-Israel p...",entailment,tensor(-9.4390),tensor(-6.6850),False
6,"singer and actress Britney Spears, 24, has fil...",entailment,tensor(-8.4483),tensor(-6.2529),False
7,Following the successful bid to bring the 2010...,entailment,tensor(-8.5244),tensor(-6.5211),False
8,Steve Jobs was attacked by Sculley and other A...,entailment,tensor(-7.2638),tensor(-6.2313),False
9,"Traditionally, the Brahui of the Raisani tribe...",entailment,tensor(-8.8966),tensor(-6.5264),False


In [7]:
# export to csv
from google.colab import drive
drive.mount('/content/drive')

df.to_csv('/content/drive/My Drive/RLProject/NLI_llama2_7b_chat.csv', index=False)

Mounted at /content/drive


Link to csv file: https://drive.google.com/file/d/1SFLAnNKQYt__7fungMqVZt47QVKcYTtE/view?usp=sharing